In [1]:
from __future__ import annotations

from math import pi
from typing import TYPE_CHECKING

import numpy as np
import pytest
from graphix import instruction
from graphix.fundamentals import Plane
from graphix.gflow import find_flow
from graphix.opengraph import OpenGraph
from graphix.parameter import Placeholder
from graphix.random_objects import rand_circuit
from graphix.sim import DensityMatrix
from graphix.transpiler import Circuit
from graphix_jcz_transpiler import (
    CZ,
    J,
    transpile_jcz,
)
from numpy.random import Generator

from graphix_brickwork_transpiler import (
    instruction_to_jcnot,
    nqubits_from_layers,
    transpile_brickwork,
    transpile_to_layers,
)

if TYPE_CHECKING:
    from numpy.random import PCG64

TEST_BASIC_CIRCUITS = [
    Circuit(1, instr=[instruction.H(0)]),
    Circuit(1, instr=[instruction.S(0)]),
    Circuit(1, instr=[instruction.X(0)]),
    Circuit(1, instr=[instruction.Y(0)]),
    Circuit(1, instr=[instruction.Z(0)]),
    Circuit(1, instr=[instruction.I(0)]),
    Circuit(1, instr=[instruction.RX(0, pi / 4)]),
    Circuit(1, instr=[instruction.RY(0, pi / 4)]),
    Circuit(1, instr=[instruction.RZ(0, pi / 4)]),
    Circuit(2, instr=[instruction.CNOT(0, 1)]),
    Circuit(3, instr=[instruction.CCX(0, (1, 2))]),
    Circuit(2, instr=[instruction.RZZ(0, 1, pi / 4)]),
]

In [2]:
circuit = TEST_BASIC_CIRCUITS[0]
pattern_brickwork = transpile_brickwork(circuit).pattern
state_original = circuit.simulate_statevector().statevec
state_brickwork = pattern_brickwork.simulate_pattern()
dmatrix = DensityMatrix(state_original)
dmatrix_brickwork = DensityMatrix(data=state_brickwork)  # type: ignore  # noqa: PGH003
if dmatrix.nqubit != dmatrix_brickwork.nqubit:
    assert dmatrix.nqubit == dmatrix_brickwork.nqubit - 1
    dmatrix_brickwork.ptrace(dmatrix_brickwork.nqubit - 1)
assert np.abs(np.dot(dmatrix_brickwork.flatten().conjugate(), dmatrix.flatten())) == pytest.approx(1)